In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# URL for the Premier League stats
standing_url = "https://fbref.com/en/comps/9/premier-league-stats"

# Get the webpage
data = requests.get(standing_url)
soup = BeautifulSoup(data.text, 'html.parser')

# Print and check if the CSS selector is correct
standings_table = soup.select('table.stats_table')

# Check if any table is found
if standings_table:
    standings_table = standings_table[0]
    print("Table found!")
    
    # Extract team URLs
    links = standings_table.find_all('a')
    links = [l.get("href") for l in links]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]
    
    # Get data for the first team
    team_url = team_urls[0]
    data = requests.get(team_url)
    
    # Read the matches table
    matches = pd.read_html(data.text, match="Scores & Fixtures")[0]
    print(matches.head())
else:
    print("Table not found.")

# Extract team shooting statistics URLs
links = standings_table.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'all_comps/shooting/' in l]

# Get data for the first team's shooting stats
if links:
    shooting_url = f"http://fbref.com{links[0]}"
    data = requests.get(shooting_url)
    
    # Read the shooting stats table
    shooting = pd.read_html(data.text, match="Shooting")[0]
    print("shooting.")
else:
    print("Shooting stats not found.")


Table found!
         Date   Time              Comp                Round  Day    Venue  \
0  2023-08-06  16:00  Community Shield  FA Community Shield  Sun  Neutral   
1  2023-08-11  20:00    Premier League          Matchweek 1  Fri     Away   
2  2023-08-16  22:00         Super Cup       UEFA Super Cup  Wed     Home   
3  2023-08-19  20:00    Premier League          Matchweek 2  Sat     Home   
4  2023-08-27  14:00    Premier League          Matchweek 3  Sun     Away   

  Result     GF     GA       Opponent   xG  xGA  Poss  Attendance  \
0      D  1 (1)  1 (4)        Arsenal  NaN  NaN    55     81145.0   
1      W      3      0        Burnley  1.9  0.3    65     21572.0   
2      D  1 (5)  1 (4)     es Sevilla  NaN  NaN    74         NaN   
3      W      1      0  Newcastle Utd  1.0  0.3    59     53419.0   
4      W      2      1  Sheffield Utd  3.5  0.7    79     31336.0   

           Captain Formation            Referee  Match Report  \
0      Kyle Walker   4-2-3-1     Stuart Attw

C:\Users\USER\AppData\Local\Temp\ipykernel_8056\3468222654.py:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  matches = pd.read_html(data.text, match="Scores & Fixtures")[0]


In [2]:
matches.head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,xG,xGA,Poss,Attendance,Captain,Formation,Referee,Match Report,Notes
0,2023-08-06,16:00,Community Shield,FA Community Shield,Sun,Neutral,D,1 (1),1 (4),Arsenal,NaN,NaN,55,81145.0,Kyle Walker,4-2-3-1,Stuart Attwell,Match Report,Arsenal won on penalty kicks following normal ...
1,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,1.9,0.3,65,21572.0,Kevin De Bruyne,4-2-3-1,Craig Pawson,Match Report,NaN
2,2023-08-16,22:00,Super Cup,UEFA Super Cup,Wed,Home,D,1 (5),1 (4),es Sevilla,NaN,NaN,74,NaN,Kyle Walker,4-2-3-1,François Letexier,Match Report,NaN
3,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,1.0,0.3,59,53419.0,Kyle Walker,4-2-3-1,Robert Jones,Match Report,NaN
4,2023-08-27,14:00,Premier League,Matchweek 3,Sun,Away,W,2,1,Sheffield Utd,3.5,0.7,79,31336.0,Kyle Walker,4-2-3-1,Jarred Gillett,Match Report,NaN


In [3]:
shooting = pd.read_html(data.text, match="Shooting")[0]
shooting.columns = shooting.columns.get_level_values(1)
shooting.head()

C:\Users\USER\AppData\Local\Temp\ipykernel_8056\3043354422.py:1: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  shooting = pd.read_html(data.text, match="Shooting")[0]


,Player,Nation,Pos,Age,90s,Gls,Sh,SoT,SoT%,Sh/90,...,Dist,FK,PK,PKatt,xG,npxG,npxG/Sh,G-xG,np:G-xG,Matches
0,Rodri,es ESP,MF,27.0,32.6,8,62,22,35.5,1.90,...,20.6,0,0,0,4.1,4.1,0.07,3.9,3.9,Matches
1,Phil Foden,eng ENG,"FW,MF",23.0,31.7,19,105,48,45.7,3.31,...,18.5,4,0,0,10.3,10.3,0.10,8.7,8.7,Matches
2,Ederson,br BRA,GK,29.0,30.9,0,0,0,NaN,0.00,...,NaN,0,0,0,0.0,0.0,NaN,0.0,0.0,Matches
3,Julián Álvarez,ar ARG,"MF,FW",23.0,29.4,11,96,37,38.5,3.26,...,17.8,13,2,2,13.0,11.5,0.12,-2.0,-2.5,Matches
4,Kyle Walker,eng ENG,DF,33.0,30.7,0,16,3,18.8,0.52,...,29.4,1,0,0,0.4,0.4,0.02,-0.4,-0.4,Matches


In [4]:
team_data = matches.merge(shooting[["xG", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="xG")
team_data

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,Match Report,NaN,15,5,17.0,0,0,0
1,2023-11-04,15:00,Premier League,Matchweek 11,Sat,Home,W,6,1,Bournemouth,...,3-4-3,Craig Pawson,Match Report,NaN,15,5,17.0,0,0,0
2,2024-04-06,12:30,Premier League,Matchweek 32,Sat,Away,W,4,2,Crystal Palace,...,3-2-4-1,Paul Tierney,Match Report,NaN,15,5,17.0,0,0,0
3,2024-05-19,16:00,Premier League,Matchweek 38,Sun,Home,W,3,1,West Ham,...,4-1-4-1,John Brooks,Match Report,NaN,15,5,17.0,0,0,0
4,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,Match Report,NaN,4,2,15.9,0,0,0
5,2024-03-31,16:30,Premier League,Matchweek 30,Sun,Home,D,0,0,Arsenal,...,4-1-4-1,Anthony Taylor,Match Report,NaN,4,2,15.9,0,0,0
6,2023-09-19,20:00,Champions Lg,Group stage,Tue,Home,W,3,1,rs Red Star,...,4-2-3-1,João Pinheiro,Match Report,NaN,62,22,20.6,0,0,0
7,2023-09-23,15:00,Premier League,Matchweek 6,Sat,Home,W,2,0,Nott'ham Forest,...,4-2-3-1,Anthony Taylor,Match Report,NaN,22,5,20.8,0,0,0
8,2023-11-25,12:30,Premier League,Matchweek 13,Sat,Home,D,1,1,Liverpool,...,3-2-4-1,Chris Kavanagh,Match Report,NaN,22,5,20.8,0,0,0
9,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Home,W,2,1,Brighton,...,3-2-4-1,Robert Jones,Match Report,NaN,9,4,17.9,0,0,0


In [5]:
years = list(range(2023, 2021, -1))
print(years)

[2023, 2022]


In [6]:
all_matches = []

In [7]:
standing_url = "https://fbref.com/en/comps/9/premier-league-stats"

In [13]:
for year in years:
    data = requests.get(standing_url)
    soup = BeautifulSoup(data.text, 'html.parser')
    standings_table = soup.select('table.stats_table')

    years = list(range(2023, 2021, -1))
    print(years)
    
if standings_table:
    standings_table = standings_table[0]
    print("Table found!")
else:
    print("Table not found")  
    
    links = [l.get("href") for l in standing_tables.find_all('a')]
    links = [l for l in links if '/squads/' in l]
    team_urls = [f"http://fbref.com{l}" for l in links]
    
    previous_season = soup.select("a.prev")[0].get("href")
    standing_url = f"https://fbref.com/{previous_season}"
    
    for team_url in team_urls:
        team_name = team_url.split("/")[-1].replace("-stats", "").replace("-" "")
        
        data = requests.get(team_urls)
        matches = pd.read_html(data.text, match="Scores & Fixtures")
        
        soup = BeautifulSoup(data.text)
        links = [l.get("href") for l in soup.find_all('a')]
        links = [l for l in links if l and 'all_comps/shooting/' in l]
        data = request.get(f"https://fbref.com{links[o]}")
        shooting = pd.read_html(html, match="Shooting")[0]
        shooting.columns = shooting.columns.droplevel()
        
        try:
            team_data = matches.merge(shooting[["xG", "Sh", "SoT", "Dist", "FK", "PK", "PKatt"]], on="xG")
        except ValueError:
            continue
        
        team_data = team_data[team_data["Comp"] == "Premier League"]
        team_data["Season"] = year
        team_data["Team"] = team-name
        all_matches.append(team_data)
        time.sleep(1)        

[2023, 2022]
[2023, 2022]
Table found!


In [14]:
all_matches = []
all_matches.append(team_data)
match_df = pd.concat(all_matches) 

In [15]:
match_df

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,Formation,Referee,Match Report,Notes,Sh,SoT,Dist,FK,PK,PKatt
0,2023-08-11,20:00,Premier League,Matchweek 1,Fri,Away,W,3,0,Burnley,...,4-2-3-1,Craig Pawson,Match Report,NaN,15,5,17.0,0,0,0
1,2023-11-04,15:00,Premier League,Matchweek 11,Sat,Home,W,6,1,Bournemouth,...,3-4-3,Craig Pawson,Match Report,NaN,15,5,17.0,0,0,0
2,2024-04-06,12:30,Premier League,Matchweek 32,Sat,Away,W,4,2,Crystal Palace,...,3-2-4-1,Paul Tierney,Match Report,NaN,15,5,17.0,0,0,0
3,2024-05-19,16:00,Premier League,Matchweek 38,Sun,Home,W,3,1,West Ham,...,4-1-4-1,John Brooks,Match Report,NaN,15,5,17.0,0,0,0
4,2023-08-19,20:00,Premier League,Matchweek 2,Sat,Home,W,1,0,Newcastle Utd,...,4-2-3-1,Robert Jones,Match Report,NaN,4,2,15.9,0,0,0
5,2024-03-31,16:30,Premier League,Matchweek 30,Sun,Home,D,0,0,Arsenal,...,4-1-4-1,Anthony Taylor,Match Report,NaN,4,2,15.9,0,0,0
6,2023-09-19,20:00,Champions Lg,Group stage,Tue,Home,W,3,1,rs Red Star,...,4-2-3-1,João Pinheiro,Match Report,NaN,62,22,20.6,0,0,0
7,2023-09-23,15:00,Premier League,Matchweek 6,Sat,Home,W,2,0,Nott'ham Forest,...,4-2-3-1,Anthony Taylor,Match Report,NaN,22,5,20.8,0,0,0
8,2023-11-25,12:30,Premier League,Matchweek 13,Sat,Home,D,1,1,Liverpool,...,3-2-4-1,Chris Kavanagh,Match Report,NaN,22,5,20.8,0,0,0
9,2023-10-21,15:00,Premier League,Matchweek 9,Sat,Home,W,2,1,Brighton,...,3-2-4-1,Robert Jones,Match Report,NaN,9,4,17.9,0,0,0


In [11]:
match_df.columns = [c.lower() for c in match_df.columns]

In [12]:
match_df.to_csv("matches.csv", index=False)